In [30]:
import json
import numpy as np
import wowsims

# Load Fury Raid Sim Request JSON

In [2]:
f = open('data/fury-human-bis-p3.json')
settings = json.load(f)

# Set environment and settings

In [3]:
def reset():
    wowsims.new(json.dumps(settings).encode('utf-8'))

In [4]:
# Iterations are currently capped at 3000
iterations = settings['simOptions']['iterations']

duration = settings['encounter']['duration']

In [34]:
spell_id_lists = {
        '0' : "Melee"
        '23881' : "Bloodthirst",
        '1680' : "Whirlwind",
        '47475' : "Slam",
        '47450' : "Heroic Strike",
        '47471' : "Execute",
        '12867': "Deep Wounds"
        '12292' : "Death Wish",
        '1719' : "Recklessness",
        '64382' : "Shattering Throw",
        '54758' : "Engi Gloves",
}

# Default sim agent

In [5]:
settings['simOptions']['interactive'] = False
damages = np.array([])

for i in range(iterations):
    reset()
    while not wowsims.step():
        pass
    totalDamage = wowsims.getDamageDone()
    damages = np.append(damages, totalDamage)

print(f'Average DPS: {damages.mean() / duration}')

Average DPS: 11782.084625708363


In [6]:
settings['simOptions']['interactive'] = False

while not wowsims.step():
    pass
json_string = wowsims.getSpellMetrics()
cast_metrics = json.loads(json_string)
for spell_id, metrics in cast_metrics.items():
    # Only one target, so we can just take the first one
    print(f'{spell_id}: {metrics[0]}')

0: {'Casts': 1, 'Misses': 0, 'Hits': 0, 'Crits': 0, 'Crushes': 0, 'Dodges': 0, 'Glances': 0, 'Parries': 0, 'Blocks': 0, 'TotalDamage': 0, 'TotalThreat': 0, 'TotalHealing': 0, 'TotalShielding': 0, 'TotalCastTime': 0}
12292: {'Casts': 1, 'Misses': 0, 'Hits': 0, 'Crits': 0, 'Crushes': 0, 'Dodges': 0, 'Glances': 0, 'Parries': 0, 'Blocks': 0, 'TotalDamage': 0, 'TotalThreat': 0, 'TotalHealing': 0, 'TotalShielding': 0, 'TotalCastTime': 0}
12867: {'Casts': 115, 'Misses': 0, 'Hits': 115, 'Crits': 0, 'Crushes': 0, 'Dodges': 0, 'Glances': 0, 'Parries': 0, 'Blocks': 0, 'TotalDamage': 224429.53180992126, 'TotalThreat': 161703.5672023456, 'TotalHealing': 0, 'TotalShielding': 0, 'TotalCastTime': 0}
1680: {'Casts': 13, 'Misses': 0, 'Hits': 7, 'Crits': 6, 'Crushes': 0, 'Dodges': 0, 'Glances': 0, 'Parries': 0, 'Blocks': 0, 'TotalDamage': 85506.77740753008, 'TotalThreat': 74818.43023158883, 'TotalHealing': 0, 'TotalShielding': 0, 'TotalCastTime': 19500000000}
1719: {'Casts': 1, 'Misses': 0, 'Hits': 0, 'C

# PPO Agent

## Load Trained PPO Model

In [25]:
from ray.rllib.algorithms.ppo import PPOConfig
from ray.tune.registry import register_env
from fury_sim_env import FurySimEnv
from gymnasium import make

def env_creator(env_config):
    return FurySimEnv(...)

register_env("FurySimEnv", env_creator)

config = PPOConfig()\
            .rollouts(num_rollout_workers=1)
algorithm = config.build(env="FurySimEnv")
algorithm.restore("models\Fury\PPO\PPO_FurySimEnv_2db34_00000_0_2023-04-30_16-55-46\checkpoint_000050")

<>:14: DeprecationWarning: invalid escape sequence \F
(pid=12464)   if (distutils.version.LooseVersion(tf.__version__) <
(pid=12464)   if (distutils.version.LooseVersion(tf.__version__) <
(pid=26228) c:\ProgramData\Anaconda3\envs\civ6_rl_rllib\lib\site-packages\tensorflow_probability\python\__init__.py:57: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
(pid=26228)   if (distutils.version.LooseVersion(tf.__version__) <
2023-04-30 18:23:21,825	INFO trainable.py:172 -- Trainable.setup took 12.367 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
2023-04-30 18:23:21,827	WARNING util.py:67 -- Install gputil for GPU system monitoring.
(RolloutWorker pid=27604) 2023-04-30 18:23:21,771	WARNING env.py:155 -- Your env doesn't have a .spec.max_episode_steps attribute. Your horizon will default to infinity, and your environment will not be reset.
2023-04-30 18:23:21,884	INFO trainab

## Run PPO Model

In [27]:
from gymnasium.envs.registration import register
register(id="FurySimEnv", entry_point="fury_sim_env:FurySimEnv")
env = make("FurySimEnv")
for i in range(100):
    observation, info = env.reset()
    terminated = False
    reward = 0
    batch = []
    dps_results = np.array([])
    metrics_batch = []
    while not terminated:
        action = algorithm.compute_single_action(observation)
        observation, reward, terminated, truncated, info = env.step(action)
        batch.append(np.concatenate((observation, [action], [reward])))
    dps_results = np.append(dps_results, info['dps'])
    metrics_batch.append(info['spell metrics'])

c:\ProgramData\Anaconda3\envs\civ6_rl_rllib\lib\site-packages\gymnasium\envs\registration.py:498: UserWarning: WARN: Overriding environment FurySimEnv already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")


The examples so far have only looked at damage output.
More detailed spell metrics are also available.

In [35]:
median = np.argsort(dps_results)[len(dps_results)//2]
print(f'Average DPS: {dps_results[median]}')

cast_metrics = json.loads(metrics_batch[median])
for spell_id, metrics in cast_metrics.items():
    # Only one target, so we can just take the first one
    try:
        spell_name = spell_id_lists[spell_id]
    except KeyError as key:
        spell_name = key
    print(f'{spell_name}: {metrics[0]}')

Average DPS: 12402.322180633872
'0': {'Casts': 1, 'Misses': 0, 'Hits': 0, 'Crits': 0, 'Crushes': 0, 'Dodges': 0, 'Glances': 0, 'Parries': 0, 'Blocks': 0, 'TotalDamage': 0, 'TotalThreat': 0, 'TotalHealing': 0, 'TotalShielding': 0, 'TotalCastTime': 0}
Death Wish: {'Casts': 1, 'Misses': 0, 'Hits': 0, 'Crits': 0, 'Crushes': 0, 'Dodges': 0, 'Glances': 0, 'Parries': 0, 'Blocks': 0, 'TotalDamage': 0, 'TotalThreat': 0, 'TotalHealing': 0, 'TotalShielding': 0, 'TotalCastTime': 0}
'12867': {'Casts': 124, 'Misses': 0, 'Hits': 124, 'Crits': 0, 'Crushes': 0, 'Dodges': 0, 'Glances': 0, 'Parries': 0, 'Blocks': 0, 'TotalDamage': 268282.1412079626, 'TotalThreat': 187797.49884557363, 'TotalHealing': 0, 'TotalShielding': 0, 'TotalCastTime': 0}
Whirlwind: {'Casts': 4, 'Misses': 0, 'Hits': 0, 'Crits': 0, 'Crushes': 0, 'Dodges': 0, 'Glances': 0, 'Parries': 0, 'Blocks': 0, 'TotalDamage': 0, 'TotalThreat': 0, 'TotalHealing': 0, 'TotalShielding': 0, 'TotalCastTime': 0}
Recklessness: {'Casts': 1, 'Misses': 0, 'H